### 迴歸分析的 ANOVA(變異數分析;方差) 表

製作迴歸的 ANOVA 表的目的在於，檢定自變量是否能解釋依變量，也可以用來比較同依變量不同自變量的 p值(越低代表那些自變量較能解釋依變量)。

其中，ANOVA 的 sum of squares(SS) 有分為 type I SS, type II SS 和 type III SS。
*   type I : 按照順序將變異分配給不同的變項, 例如有 A, B 兩變項及 AB 的交互作用項, 依順序放入, 則各個變項的變異分別是 SSR(A), SSR(B|A), SSR(AB|B,A). 因為順序會影響結果, 所以必須決定變異放入的順序.
*   type II : 沒有變項放入順序問題. 並且 type II 假設沒有交互作用(若檢定出來有交互作用項則無法使用 type II), 所以各變異分別為 SSR(A|B), SSR(B|A).
*   type III: 和 type II 一樣沒有變項放入順序的問題, 但此 type 會考慮有交互作用項, 所以各變異分別為 SSR(A|B, AB), SSR(B|A, AB).
*   type II 和 III 也可以說是看最後放入該變項後 SSR 的改變量.
*   [參考網站 : Anova – Type I/II/III SS explained](https://www.r-bloggers.com/2011/03/anova-%E2%80%93-type-iiiiii-ss-explained/)
*   [參考網站 : facebook](https://www.facebook.com/258972167562965/posts/2711039792356178/)

為了決定要用那些自變量，可以使用不同方法，例如 forward、backward ，又稱逐步迴歸。
*   Forward : 從完全沒有自變量開始，和所有未放入的自變量一一配對，選擇放入後決定係數最高的特徵, 重複直到決定係數上升率小於限制值(自己給定)或沒有特徵能放入.
*   Backward : 和 Forward 類似，不過是從選擇所有自變量開始一一移除。
*   Forward & Backward : 放入和刪除同時進行，選擇最高的決定係數
*   [參考網站 : Understand Forward And Backward Stepwise Regression](https://quantifyinghealth.com/stepwise-selection/)

<img src="https://imgur.com/3ygHs0q.png" width="70%" height="70%">

參考資料:
*   [【统计学笔记】方差分析表和回归分析表的解读](https://blog.csdn.net/MYMarcoreus/article/details/111945748)
*   [多元线性回归方差分析表理解](https://blog.51cto.com/u_14902625/5472080)

其他程式:
*   R : [ANOVA and model fit](https://bookdown.org/egarpor/SSS2-UC3M/multlin-aovfit.html)

### python

<font color = red size = 5>此 class 沒有考慮交互作用項</font>

In [1]:
import numpy as np
import scipy.stats as ss

class regression_ANOVA:
    def __init__(self, X, y, model):
        """
        y : 應變數
        y_hat : 預測結果
        X : 參數值

        """
        #raw data
        self.y = y.values
        self.X = X.values
        self.colnames = list(X.columns)

        self.model = model
        self.model.fit(self.X, self.y)
        self.y_hat = self.model.predict(self.X)
        self.Tl, self.Pl = np.shape(self.X)
        self.Rl = self.Tl - self.Pl - 1

    ### function
    def Forward(self):
        s = []
        while len(s) < len(self.X[0]) :
            feature_list = [i for i in range(len(self.X[0])) if not(i in s)]
            Max_i = 0
            Max_v = 0
            for i in feature_list:
                LM = self.model
                feature_i = s + [i]
                X1 = self.X[:,feature_i]

                LM.fit(X1, self.y)
                y1_pred = LM.predict(X1)
                SSR = np.sum((np.mean(self.y) - y1_pred)**2)
                if SSR > Max_v : 
                    Max_i = i
                    Max_v = SSR
            s += [Max_i]
        return s
    
    def value_code(self, x):
        if x < 0.001:
            return '***'
        if x < 0.01:
            return '**'
        if x < 0.05:
            return '*'
        if x < 0.1:
            return '.'
        else:
            return ''

    ### 
    def SST(self):
        return round(np.sum((np.mean(self.y) - self.y)**2), 3)

    def SSR(self):
        return round(np.sum((np.mean(self.y) - self.y_hat)**2), 3)

    def SSE(self):
        return round(np.sum((self.y - self.y_hat)**2), 3)

    def MSR(self):
        return round(self.SSR()/self.Pl,3)
    
    def MSE(self):
        return round(self.SSE()/self.Rl,3)
    
    def P_value(self):
        return round(1 - ss.f.cdf(self.MSR()/self.MSE(), self.Pl, self.Rl), 5)


    ### table
    def ANOVA_table(self):
        table = \
                '='*120 + '\n' +\
                '{0:^16}'.format("變異來源")  +'{0:^16}'.format("平方和(SS)") +\
                     '{0:^16}'.format("自由度(DF)") + '{0:^16}'.format("平均平方和(MS)") +\
                        '{0:^16}'.format("檢定統計量(F value)") + '{0:^16}'.format("p_value") + '\n' +\
                '-'*120 + '\n' +\
                '{0:^18}'.format("迴歸(Predictors)")+ '{0:>12}'.format(self.SSR()) +\
                     '{0:>17}'.format(self.Pl) + '{0:>22}'.format(self.MSR()) +\
                        '{0:>20}'.format(round(self.MSR()/self.MSE(),3)) + '{0:>17}'.format(self.P_value()) + '{0:>5}'.format(self.value_code(self.P_value())) +'\n' +\
                '\n' +\
                '{0:^18}'.format("殘差(Residuals)")  + '{0:>12}'.format(self.SSE()) +\
                     '{0:>17}'.format(self.Rl) + '{0:>22}'.format(self.MSE()) +\
                        '{0:>20}'.format("") + '{0:^10}'.format("") + '\n' +\
                '-'*120 +'\n' +\
                '{0:^18}'.format("總和(Total)") + '{0:>12}'.format(self.SST()) +\
                     '{0:>17}'.format(self.Tl - 1) + '{0:>22}'.format('') +\
                        '{0:^32}'.format("") + '{0:^10}'.format("") + '\n' +\
                '='*120 + '\n' +\
                "Signif. codes : 0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 '' 1 "
        print(table)


    def ANOVA_table_typeI(self):
        feature_order = self.Forward()
        ## 計算各個 SSR 的遞增值
        feature_i = []
        SSR_list = [0]

        for i in feature_order:
            LM = self.model
            feature_i.append(i)

            X1 = self.X[:,feature_i]

            LM.fit(X1, self.y)
            y1_pred = LM.predict(X1)
            SSR_list.append(round(np.sum(((np.mean(self.y) - y1_pred)**2))  - np.sum(SSR_list),3))
        SSR_list = SSR_list[1:]

        table = \
                '='*120 + '\n' +\
                '{0:^16}'.format("變異來源")  +'{0:^16}'.format("平方和(SS)") +\
                     '{0:^16}'.format("自由度(DF)") + '{0:^16}'.format("平均平方和(MS)") +\
                        '{0:^16}'.format("檢定統計量(F value)") + '{0:^16}'.format("p_value") + '\n' +\
                '-'*120 + '\n' +\
                '{0:^18}'.format("迴歸(Predictors)") + '{0:>12}'.format(self.SSR()) +\
                     '{0:>17}'.format(self.Pl) + '{0:>22}'.format(self.MSR()) +\
                        '{0:>20}'.format(round(self.MSR()/self.MSE(),3)) + '{0:>17}'.format(self.P_value()) + '{0:>5}'.format(self.value_code(self.P_value())) +'\n' +\
                '\n' 

        for i in range(len(feature_order)):
            feature_index = feature_order[i]
            F_value = round(SSR_list[i]/self.MSE(),3)
            p_value = round(1 - ss.f.cdf(F_value, 1, self.Rl), 5)
            table += '{0:>16}'.format(self.colnames[feature_index])  + '{0:>16}'.format(SSR_list[i]) +\
                     '{0:>17}'.format(1) + '{0:>22}'.format(SSR_list[i]) +\
                     '{0:>20}'.format(F_value) + '{0:>17}'.format(p_value) + '{0:>5}'.format(self.value_code(p_value)) +'\n' +\
                '\n' 

        table +=\
                '{0:^18}'.format("殘差(Residuals)")  + '{0:>12}'.format(self.SSE()) +\
                     '{0:>17}'.format(self.Rl) + '{0:>22}'.format(self.MSE()) +\
                        '{0:>20}'.format("") + '{0:^10}'.format("") + '\n' +\
                '-'*120 +'\n' +\
                '{0:^18}'.format("總和(Total)") + '{0:>12}'.format(self.SST()) +\
                     '{0:>17}'.format(self.Tl - 1) + '{0:>22}'.format('') +\
                        '{0:>20}'.format("") + '{0:^10}'.format("") + '\n' +\
                '='*120 + '\n' +\
                "Signif. codes : 0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 '' 1 "
        
        print(table)


    def ANOVA_table_typeII(self):
        feature_order = self.Forward()
        ## 計算各個 SSR 的遞增值
        feature_i = []
        SSR_list = []
        SSR = self.SSR()

        for i in feature_order:
            LM = self.model
            feature_i = [j for j in range(len(feature_order)) if j != i]
            X1 = self.X[:,feature_i]

            LM.fit(X1, self.y)
            y1_pred = LM.predict(X1)
            SSR_list.append(round(SSR - np.sum(((np.mean(self.y) - y1_pred)**2)) ,3))


        table = \
                '='*120 + '\n' +\
                '{0:^16}'.format("變異來源")  +'{0:^16}'.format("平方和(SS)") +\
                     '{0:^16}'.format("自由度(DF)") + '{0:^16}'.format("平均平方和(MS)") +\
                        '{0:^16}'.format("檢定統計量(F value)") + '{0:^16}'.format("p_value") + '\n' +\
                '-'*120 + '\n' +\
                '{0:^18}'.format("迴歸(Predictors)") + '{0:>12}'.format(self.SSR()) +\
                     '{0:>17}'.format(self.Pl) + '{0:>22}'.format(self.MSR()) +\
                        '{0:>20}'.format(round(self.MSR()/self.MSE(),3)) + '{0:>17}'.format(self.P_value()) + '{0:>5}'.format(self.value_code(self.P_value())) +'\n' +\
                '\n' 

        for i in range(len(feature_order)):
            feature_index = feature_order[i]
            F_value = round(SSR_list[i]/self.MSE(),3)
            p_value = round(1 - ss.f.cdf(F_value, 1, self.Rl), 5)
            table += '{0:>16}'.format(self.colnames[feature_index])  + '{0:>16}'.format(SSR_list[i]) +\
                     '{0:>17}'.format(1) + '{0:>22}'.format(SSR_list[i]) +\
                     '{0:>20}'.format(F_value) + '{0:>17}'.format(p_value) + '{0:>5}'.format(self.value_code(p_value)) +'\n' +\
                '\n' 
        

        table +=\
                '{0:^18}'.format("殘差(Residuals)")  + '{0:>12}'.format(self.SSE()) +\
                     '{0:>17}'.format(self.Rl) + '{0:>22}'.format(self.MSE()) +\
                        '{0:>20}'.format("") + '{0:^10}'.format("") + '\n' +\
                '-'*120 +'\n' +\
                '{0:^18}'.format("總和(Total)") + '{0:>12}'.format(self.SST()) +\
                     '{0:>17}'.format(self.Tl - 1) + '{0:>22}'.format('') +\
                        '{0:>20}'.format("") + '{0:^10}'.format("") + '\n' +\
                '='*120 + '\n' +\
                "Signif. codes : 0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 '' 1 "
        
        print(table)



In [2]:
import pandas as pd
from sklearn.linear_model import LinearRegression
# 從 CSV 檔案讀取資料
#boston_data = pd.read_csv('BostonHousing.csv')

# 分割特徵和目標變數
#X = boston_data.drop('medv', axis=1)  # 假設目標變數的欄位名稱為 'target_column'
#y = boston_data['medv']

data = pd.read_csv('mranova_test.csv', index_col = 'i') 

X = data.iloc[:,1:]
y = data.iloc[:,0]
Model = LinearRegression()



In [3]:
# anova table
anova = regression_ANOVA(X, y, Model)
anova.ANOVA_table()

      變異來源          平方和(SS)         自由度(DF)        平均平方和(MS)     檢定統計量(F value)     p_value     
------------------------------------------------------------------------------------------------------------------------
  迴歸(Predictors)      5983.363                5              1196.673            1180.151              0.0  ***

  殘差(Residuals)         95.269               94                 1.014                              
------------------------------------------------------------------------------------------------------------------------
    總和(Total)         6078.632               99                                                                
Signif. codes : 0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 '' 1 


In [4]:
# type I
anova = regression_ANOVA(X, y, Model)
anova.ANOVA_table_typeI()

      變異來源          平方和(SS)         自由度(DF)        平均平方和(MS)     檢定統計量(F value)     p_value     
------------------------------------------------------------------------------------------------------------------------
  迴歸(Predictors)      5983.363                5              1196.673            1180.151              0.0  ***

               v        3104.419                1              3104.419            3061.557              0.0  ***

               u        1703.886                1              1703.886            1680.361              0.0  ***

               e         738.965                1               738.965             728.762              0.0  ***

               w         434.811                1               434.811             428.808              0.0  ***

               d           1.282                1                 1.282               1.264          0.26376     

  殘差(Residuals)         95.269               94                 1.014                         

In [5]:
# type II
anova = regression_ANOVA(X, y, Model)
anova.ANOVA_table_typeII()

      變異來源          平方和(SS)         自由度(DF)        平均平方和(MS)     檢定統計量(F value)     p_value     
------------------------------------------------------------------------------------------------------------------------
  迴歸(Predictors)      5983.363                5              1196.673            1180.151              0.0  ***

               v        2657.318                1              2657.318            2620.629              0.0  ***

               u        1744.852                1              1744.852            1720.761              0.0  ***

               e          769.62                1                769.62             758.994              0.0  ***

               w         423.484                1               423.484             417.637              0.0  ***

               d           1.282                1                 1.282               1.264          0.26376     

  殘差(Residuals)         95.269               94                 1.014                         